In [1]:
import os
import sys

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
import math
import random
import wandb

from tqdm.notebook import tqdm
from multiprocess import set_start_method

import torch
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset, load_from_disk, Audio, concatenate_datasets
from datasets import Value

import transformers
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM,  
    TrainingArguments, 
    default_data_collator, 
    get_scheduler
)
from accelerate import Accelerator
import bitsandbytes as bnb

from peft import LoraConfig, get_peft_model

import dac
from audiotools import AudioSignal

/opt/conda/envs/4o/lib/python3.12/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
!nvidia-smi

Wed Aug 14 14:26:33 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:04:00.0 Off |                  Off |
| N/A   41C    P0    58W / 250W |  25948MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                  Off |
| N/A   

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
base_model = "google/gemma-2-2b"
save_dir = "./results"

start_audio_token = "<soa>"
end_audio_token = "<eos>"
end_frame_token = "<eof>"

n_codebooks = 8
max_seq_length = 1024

device = 0
load_processed = True
path_to_processed = "./data/processed/"
path_to_cache = "./data/cache/"

torch.cuda.set_device(f"cuda:{device}")

In [6]:
lora_config = LoraConfig(
    r=16,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

In [7]:
class Vikhr4oDataset(Dataset):
    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer

        self.start_audio_token = tokenizer(start_audio_token, return_tensors="pt")["input_ids"][:, 1:]
        self.end_audio_token = tokenizer(end_audio_token, return_tensors="pt")["input_ids"][:, 1:]
        self.end_frame_token = tokenizer(end_frame_token, return_tensors="pt")["input_ids"][:, 1:].item()

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        row = self.dataset[idx]
        text = row["text"]
        text_tokenized = self.tokenizer(text, return_tensors="pt")
        text_input_tokens = text_tokenized["input_ids"]
        
        raw_audio_tokens = torch.tensor(row["audio_tokens"])[:, :n_codebooks]
        n_frames = raw_audio_tokens.shape[-1]
        raw_audio_tokens = torch.cat([raw_audio_tokens, torch.full((1, 1, n_frames), self.end_frame_token)], dim=1)
        
        # permute: (n_codebooks, n_frames) -> (n_frames, n_codebooks)
        audio_input_tokens = raw_audio_tokens.permute(2, 0, 1).contiguous().view(1, -1)

        audio_length = min(max_seq_length - text_input_tokens.shape[-1] - 2, audio_input_tokens.shape[-1])
        audio_length -= audio_length % (n_codebooks + 1)

        padding_size = max_seq_length - text_input_tokens.shape[-1] - audio_length - 2
        padding = torch.zeros((1, padding_size), dtype=torch.int64)
        
        tokens = torch.cat([padding, text_input_tokens, self.start_audio_token, audio_input_tokens[:, :audio_length], self.end_audio_token], dim=1).squeeze(0)
        attention_mask = torch.cat([padding, torch.ones((1, max_seq_length - padding_size))], dim=1).squeeze(0)

        return {
            "input_ids": tokens, 
            "attention_mask": attention_mask, 
            "labels": tokens.clone(),
        }

In [8]:
tokenizer = AutoTokenizer.from_pretrained(base_model, cache_dir=".")
model = AutoModelForCausalLM.from_pretrained(base_model,
                                             device_map={"":0}, 
                                             attn_implementation="eager",
                                             cache_dir=".")

# if tokenizer.model_max_length > 100_000:
#     tokenizer.model_max_length = max_seq_length

n_tokens = len(tokenizer)
tokenizer.add_special_tokens({'additional_special_tokens': [start_audio_token]})
start_audio_token_id = tokenizer(start_audio_token)["input_ids"][-1]

quant_path = dac.utils.download(model_type="16khz")
quantizer = dac.DAC.load(quant_path, n_codebooks=n_codebooks).to(f"cpu")
tokenizer.add_tokens([f"<audio_token_{i}>" for i in range(quantizer.codebook_size)])
tokenizer.add_special_tokens({'additional_special_tokens': [end_frame_token]})
end_frame_token_id = tokenizer(end_frame_token)["input_ids"][-1]

assert len(tokenizer) == n_tokens + 2 + quantizer.codebook_size
model.resize_token_embeddings(len(tokenizer))

model = get_peft_model(model, lora_config)

# fine-tune embedings, since they are randomly initialised for new tokens
model.base_model.model.model.embed_tokens.weight.requires_grad = True

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/opt/conda/envs/4o/lib/python3.12/site-packages/audiotools/ml/layers/base.py:172: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_dict = torch.load(location, "cpu")
/opt

In [9]:
def quantize(example, rank, quantizer, n_original_tokens):
    # repeat import to work with multiprocessing
    import torch

    quantizer.to(f"cuda:{(rank or 0) % torch.cuda.device_count()}")
    audio_data, sample_rate = example["audio"]["array"], example["audio"]["sampling_rate"]

    # audio -> discrete codes
    audio = torch.tensor(audio_data).view(1, 1, len(audio_data)).float()
    audio = audio.to(quantizer.device)
    x = quantizer.preprocess(audio, sample_rate)
    _, codes, _, _, _ = quantizer.encode(x)

    # Move tensor back to CPU and delete it to free GPU memory
    del audio
    del x
    torch.cuda.empty_cache()

    # increment tokens' ids 
    return {"audio_tokens": codes + n_original_tokens}


def get_audio_padding_tokens(quantizer, n_original_tokens):
    # create audio without any sounds 
    # seems to work better than radom padding if 
    # length of generated audio is not devisible by n_codebooks
    audio = torch.zeros((1, 1, 1))
    audio = audio.to(quantizer.device)
    
    x = quantizer.preprocess(audio, quantizer.sample_rate)
    _, codes, _, _, _ = quantizer.encode(x)

    # Move tensor back to CPU and delete it to free GPU memory
    del audio
    del x
    torch.cuda.empty_cache()
    
    return {"audio_tokens": codes[0].t()}
    


def decode_audio(tokens, quantizer, pad_tokens, n_original_tokens):
    # find start and end indices of audio tokens 
    tokens = tokens[tokens != end_frame_token_id]
    start = torch.nonzero(tokens == start_audio_token_id)
    end = torch.nonzero(tokens == tokenizer.eos_token_id)
    
    start = start[0, -1] + 1 if len(start) else 0
    end = end[0, -1] if len(end) else tokens.shape[-1]
    # substract length of original vocabulary -> tokens in range [0, 1024)
    audio_tokens = tokens[start:end] % n_original_tokens
    reminder = audio_tokens.shape[-1] % n_codebooks
    
    if reminder:
        # pad if last frame is incomplete 
        audio_tokens = torch.cat([audio_tokens, pad_tokens[:, reminder:]], dim=1)

    codes = audio_tokens.view(1, -1, n_codebooks).permute(0, 2, 1).to(quantizer.device)
    z = quantizer.quantizer.from_codes(codes)[0]
    audio = quantizer.decode(z)

    del tokens 
    del audio_tokens 
    torch.cuda.empty_cache()
    
    return AudioSignal(audio.detach().cpu().numpy(), quantizer.sample_rate)

    

In [10]:
def prepare_librispeech():
    raw = load_dataset("openslr/librispeech_asr", "clean", cache_dir=".")
    processed = raw.remove_columns(["chapter_id"])
    processed = processed.cast_column('speaker_id', Value('string'))
    return processed 


def prepare_tedlium():
    raw = load_dataset("LIUM/tedlium", "release1", cache_dir=".")
    processed = raw.remove_columns(["gender"])
    return processed
    

if not load_processed:
    print("Loadiing TEDLIUM")
    dataset = prepare_tedlium()

    train_data = dataset["train"]
    val_data = dataset["validation"]
else:
    train_data = load_from_disk(os.path.join(path_to_processed, "train"))
    val_data = load_from_disk(os.path.join(path_to_processed, "val"))

In [11]:
if __name__ == "__main__" and not load_processed:
    set_start_method("spawn")
    train_data = train_data.map(quantize,
                                with_rank=True,
                                num_proc=torch.cuda.device_count(),
                                fn_kwargs={"quantizer": quantizer, "n_original_tokens": n_tokens + 2},
                                remove_columns=["audio", "speaker_id", "file"],
                                desc="Tokenizing train audio",
                                # cache_file_name="./data/train/cache-a4a404b29010dba0.arrow"
                                cache_file_name=os.path.join(path_to_cache, "train.arrow"),
                               )
    val_data = val_data.map(quantize,
                            with_rank=True,
                            num_proc=torch.cuda.device_count(),
                            fn_kwargs={"quantizer": quantizer, "n_original_tokens": n_tokens + 2},
                            remove_columns=["audio", "speaker_id", "file"],
                            desc="Tokenizing validation audio",
                            # cache_file_name="./data/val/cache-357a9e761ae352d3.arrow"
                            cache_file_name=os.path.join(path_to_cache, "val.arrow"),
                           )
    train_data.save_to_disk(os.path.join(path_to_processed, "train"))
    val_data.save_to_disk(os.path.join(path_to_processed, "val"))


In [12]:
train_dataset = Vikhr4oDataset(train_data, tokenizer)
val_dataset = Vikhr4oDataset(val_data, tokenizer)

padding_tokens = get_audio_padding_tokens(quantizer, n_tokens + 1)["audio_tokens"]

In [32]:
# test audio decoding

input_ids_test = val_dataset[1]["input_ids"].unsqueeze(0)
decoded = decode_audio(input_ids_test, quantizer, padding_tokens, n_tokens + 1)
noise = decode_audio(torch.zeros(input_ids_test.size(), dtype=torch.int64) + 256421, quantizer, padding_tokens, n_tokens + 1)

decoded.write("tests/test.wav")
noise.write("tests/noise.wav")

## No SFT Trainer

In [ ]:
train_batch_size = 1
eval_batch_size = 2
learning_rate = 2e-4
gradient_accumulation_steps = 8
lr_scheduler_type = "linear"
num_train_epochs = 3
num_warmup_steps = 10
checkpointing_steps = 100
logging_steps = 20
weight_decay = 0
max_grad_norm = 1



def test_audio_generation(model, batch, n, quantizer, pad_tokens, n_original_tokens):
    inds = random.choices(range(len(batch)), k=n)
    audios = []
    
    for input_ids, attn in batch["input_ids"], batch["attention_mask"]:
        with torch.no_grad():
            ind = torch.nonzero(input_ids == start_audio_token_id)[0, -1]
            input_ids = input_ids[:ind+1].unsqueeze(0)
            attn = attn[:ind+1].unsqueeze(0).to(torch.float16)
            output = model.generate(input_ids=input_ids, attention_mask=attn ,max_length=max_seq_length)

        try:
            audio = decode_audio(output, quantizer, pad_tokens, n_original_tokens)
            audio_file = os.path.join(save_dir, "audio")
            os.makedirs(audio_file, exists_ok=True)
            audio_file = os.path.join(audio_file, f"audio_{ind + 1}.wav")
            aduio.write(audio_file)
            audios.append(audio_file)
        except:
            print("No audio generated.")
            pass

    return audios


def get_last_checkpoint():
    n_checkpoints = len(list(filter(lambda x: x.startswith("checkpoint"), os.listdir(save_dir))))
    return n_checkpoints + 1


def save_checkpoint(model, accelerator, tokenizer, optimizer, scheduler):
    accelerator.wait_for_everyone()
    state = model.state_dict()

    path = os.path.join(save_dir, f"checkpoint-{get_last_checkpoint() * checkpointing_steps}")
    
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(
        path, 
        state_dict=state, 
        is_main_process=accelerator.is_main_process, 
        save_function=accelerator.save, 
        save_embedding_layers=True
    )
    if accelerator.is_main_process:
        tokenizer.save_pretrained(path)
        torch.save(optimizer.state_dict(), os.path.join(path, "optimizer.pt"))
        torch.save(scheduler.state_dict(), os.path.join(path, "scheduler.pt"))


def train(model, dataloader, accelerator, optimizer, lr_scheduler, completed_steps, progress_bar, max_train_steps):
    model.train()
    total_loss = 0
    acc_loss = 0
    
    for step, batch in enumerate(dataloader): 
        with accelerator.accumulate(model):
            outputs = model(**batch)
            loss = outputs.loss
    
            last_loss = loss.detach().float()
            total_loss += last_loss
            acc_loss += last_loss 
            
            accelerator.backward(loss)
    
        if accelerator.sync_gradients:
            accelerator.clip_grad_norm_(model.parameters(), max_grad_norm)
            
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
    
            progress_bar.update(1)
            completed_steps += 1

            acc_loss = acc_loss / gradient_accumulation_steps

            accelerator.log({"loss": acc_loss.item()})
            acc_loss = 0
    
            if completed_steps % checkpointing_steps == 0:
                save_checkpoint(model, accelerator, tokenizer, optimizer, lr_scheduler)

            torch.cuda.empty_cache()

        if completed_steps >= max_train_steps:
                break

    return total_loss / len(dataloader), completed_steps


def eval(model, dataloader, accelerator, epoch, completed_steps, train_loss, quantizer, pad_tokens, n_original_tokens):
    model.eval()
    losses = []

    eval_progress_bar = tqdm(dataloader, desc=f"Evaluating Epoch {epoch}", leave=False)
    
    for batch in eval_progress_bar:
        with torch.no_grad():
            outputs = model(**batch)

        loss = outputs.loss
        losses.append(accelerator.gather_for_metrics(loss.repeat(eval_batch_size)))

    losses = torch.cat(losses)
    try:
        eval_loss = torch.mean(losses)
        perplexity = math.exp(eval_loss)
    except OverflowError:
        perplexity = float("inf")

    print(f"epoch {epoch}: perplexity: {perplexity} eval_loss: {eval_loss}")
    # audios = test_audio_generation(model, batch, 2, quantizer, pad_tokens, n_original_tokens)

    base_log = {
        "perplexity": perplexity,
        "eval_loss": eval_loss,
        "train_loss": train_loss.item() / len(train_dataloader),
        "epoch": epoch,
        "step": completed_steps,
    }
    # base_log.update({f"audio_{i+1}": audios[i] for i in range(len(audios))})

    accelerator.log(base_log, step=completed_steps)


if __name__ == "__main__":
    accelerator = Accelerator(gradient_accumulation_steps=gradient_accumulation_steps, 
                              mixed_precision='no', 
                              log_with="wandb")
    quantizer.to(model.device)
    os.makedirs(save_dir, exist_ok=True)

    train_dataloader = DataLoader(
        train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=train_batch_size
    )
    eval_dataloader = DataLoader(
        val_dataset, collate_fn=default_data_collator, batch_size=eval_batch_size
    )

    no_decay = ["bias", "layer_norm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay) and p.requires_grad],
            "weight_decay": weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay) and p.requires_grad],
            "weight_decay": 0.0,
        },
    ]
    optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=learning_rate, fused=True)
    # optimizer = bnb.optim.Adam8bit(optimizer_grouped_parameters, min_8bit_size=16384)

    num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
    max_train_steps = num_train_epochs * num_update_steps_per_epoch

    lr_scheduler = get_scheduler(
        name=lr_scheduler_type,
        optimizer=optimizer,
        num_warmup_steps=num_warmup_steps * accelerator.num_processes,
        num_training_steps=max_train_steps * accelerator.num_processes,
    )

    model, optimizer, train_dataloader, eval_dataloader, lr_scheduler = accelerator.prepare(
        model, optimizer, train_dataloader, eval_dataloader, lr_scheduler
    )

    num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
    max_train_steps = num_train_epochs * num_update_steps_per_epoch
    
    num_train_epochs = math.ceil(max_train_steps / num_update_steps_per_epoch)

    accelerator.init_trackers("vikhr4o-gemma-2", {"lr_scheduler_type": lr_scheduler_type})

    total_batch_size = train_batch_size * accelerator.num_processes * gradient_accumulation_steps

    print("***** Running training *****")
    print(f"  Num examples = {len(train_dataset)}")
    print(f"  Num Epochs = {num_train_epochs}")
    print(f"  Instantaneous batch size per device = {train_batch_size}")
    print(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
    print(f"  Gradient Accumulation steps = {gradient_accumulation_steps}")
    print(f"  Total optimization steps = {max_train_steps}")
    
    progress_bar = tqdm(range(max_train_steps), disable=not accelerator.is_local_main_process)
    completed_steps = 0
    starting_epoch = 0

    for epoch in range(starting_epoch, num_train_epochs):
        train_loss, completed_steps = train(model, train_dataloader, accelerator, optimizer, lr_scheduler, completed_steps, progress_bar, max_train_steps)
        print(f"EPOCH {epoch + 1} train loss:", train_loss)
        eval(model, eval_dataloader, accelerator, epoch, completed_steps, train_loss, quantizer, padding_tokens, n_tokens + 1)

    save_checkpoint(model, accelerator, tokenizer, optimizer, lr_scheduler)

## SFT Trainer

In [ ]:
from trl import SFTTrainer, SFTConfig


training_args = TrainingArguments(
    learning_rate=2e-4,
    lr_scheduler_type="linear",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    eval_strategy="steps",
    eval_steps=500,
    save_steps=100, 
    save_strategy="steps",
    logging_steps=20,
    optim="paged_adamw_8bit",
    warmup_steps=10,
    output_dir="./results"
)

trainer = SFTTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        packing=True,
        max_seq_length=tokenizer.model_max_length,
    )

In [ ]:
trainer.train()